In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
class Borrower:
    """
    Represents a borrower (consumer household) in the credit scoring model.
    The borrower experiences income fluctuations and makes financial decisions.
    """
    def __init__(self, params):
        """
        Initializes a borrower with given economic model parameters.
        """
        self.beta = params.get('beta', 0.9)  # Time discount factor
        self.gamma = params.get('gamma', 2.0)  # Risk aversion coefficient
        self.rho = params.get('rho', 0.85)  # Income persistence
        self.sigma = params.get('sigma', 0.15)  # Income volatility
        self.initial_wealth = params.get('initial_wealth', 2.0)  # Initial wealth
        self.borrowing_limit = params.get('borrowing_limit', 1.0)  # Borrowing constraint
        self.interest_rate = params.get('interest_rate', 1.01)  # Initial interest rate
        
        # State variables
        self.wealth = self.initial_wealth  # Initial wealth level
        self.y = 0.0  # Initial income level
        self.consumption = 0.0  # Initial consumption level
        self.credit_history = []  # Record of past income levels for credit scoring
    
    def update_income(self):
        """
        Updates income using an AR(1) process:
        y_{t+1} = rho * y_t + sigma * epsilon
        """
        epsilon = np.random.normal(0, 1)
        self.y = self.rho * self.y + self.sigma * epsilon
    
    def compute_credit_score(self):
        """
        Computes a credit score based on wealth and past income levels.
        """
        past_income_effect = np.mean(self.credit_history[-5:]) if len(self.credit_history) >= 5 else 0
        return max(300, min(850, 700 + 5 * (self.wealth - 2) + 2 * past_income_effect))
    
    def update_wealth(self):
        """
        Updates wealth based on consumption and income.
        w_{t+1} = r_t (w_t - c_t) + e^{y_t}
        """
        r_t = self.compute_interest_rate()
        self.wealth = r_t * (self.wealth - self.consumption) + np.exp(self.y)
        self.credit_history.append(self.y)  # Store income for credit score calculations
    
    def compute_interest_rate(self):
        """
        Determines the borrowing interest rate based on the credit score.
        Higher credit scores result in lower interest rates.
        """
        credit_score = self.compute_credit_score()
        return 1.005 if credit_score > 750 else 1.02 if credit_score > 650 else 1.05

In [ ]:
class CreditScoringModel:
    """
    Represents a system that manages multiple borrowers in the credit scoring model.
    """
    def __init__(self, num_borrowers, params):
        """
        Initializes the credit scoring system with multiple borrowers.
        """
        self.borrowers = [Borrower(params=params) for _ in range(num_borrowers)]
    
    def simulate(self, periods=10):
        """
        Runs the simulation over a specified number of time periods.
        """
        results = []
        for t in range(periods):
            for borrower in self.borrowers:
                borrower.update_income()
                borrower.update_wealth()
                results.append((t, borrower.y, borrower.wealth, borrower.compute_credit_score(), borrower.compute_interest_rate()))
        return pd.DataFrame(results, columns=['time', 'income', 'wealth', 'credit_score', 'interest_rate'])

In [ ]:
def calculate_assets(self):
    """
    Calculate assets (a_t) according to a_t = w_t - c_t
    """
    return self.wealth - self.consumption